In [53]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from sklearn.model_selection import train_test_split
import transformers
import datasets
from datasets import Dataset
from transformers import pipeline

In [73]:
df = pd.read_csv("yelp.csv")
df_bert = df[["text", "stars"]]

In [74]:
df_bert["stars"] = df_bert["stars"].apply(lambda x:1 if x in [4,5] else (0 if x in [1,2] else 3))
df_bert.drop(df_bert[df_bert.stars == 3].index, inplace = True)

In [75]:
df_bert

,text,stars
0,My wife took me here on my birthday for breakf...,1
1,I have no idea why some people give bad review...,1
2,love the gyro plate. Rice is so good and I als...,1
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",1
4,General Manager Scott Petello is a good egg!!!...,1
...,...,...
9994,Let's see...what is there NOT to like about Su...,1
9996,Should be called house of deliciousness!\n\nI ...,1
9997,I recently visited Olive and Ivy for business ...,1
9998,My nephew just moved to Scottsdale recently so...,0


In [76]:
df_bert.shape

(8539, 2)

In [15]:
def tokenize_function(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
sub_train_X = train_x.shuffle(seed=123).select(range(1000))
sub_train_y = train_y.shuffle(seed=123).select(range(1000))

## Fine-tuning with Kears

### Tokenizing Dataset

In [9]:
# tokenizer for the pretrained distilbert
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [187]:
train_encodings = tokenizer(train_x, truncation = True, padding = True)
test_encodings = tokenizer(test_x, truncation = True, padding = True)

In [190]:
import tensorflow as tf
import datasets
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/338M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it fo

In [178]:
train_df = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings), 
    train_y
))

test_df = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_y
))

In [179]:
from transformers import TFAutoModelForSequenceClassification, TFTrainer, TFTrainingArguments

Training_args = TFTrainingArguments(
    output_dir ='./results',
    num_train_epochs = 4, 
    per_device_train_batch_size = 10, 
    per_device_eval_batch_size = 16, 
    warmup_steps = 500, 
    weight_decay = 0.01, 
    logging_dir = './logs', 
    logging_steps = 10,
    evaluation_strategy = 'no')

In [180]:
with Training_args.strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_79', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [183]:
trainer = TFTrainer(model = model,
                    args = Training_args,
                    train_dataset = train_df,
                    eval_dataset = test_df)
with tf.device('/GPU:0'):
    trainer.train()

2022-01-16 20:47:12.356792: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 2732
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:34"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 512
        }
      }
      shape {
        dim {
          size: 512
        }
      }
      shape {
      }
    }
  }
}

2022-01-16 20:47:13.113186: W tensorflow/core/framework/dataset.cc:744] Input of Gen

TypeError: '>' not supported between instances of 'NoneType' and 'int'